In [26]:
import cifar10
from cifar10 import num_classes
cifar10.maybe_download_and_extract()
from keras.preprocessing.image import ImageDataGenerator
import ipynb.fs.full.Structural_Perturbations as SP
import numpy as np
import tensorflow as tf

Data has apparently already been downloaded and unpacked.


In [27]:
train_data,train_labels,onehot_train=cifar10.load_training_data()
test_data,test_labels,onehot_test=cifar10.load_test_data()
train_data=train_data.astype('float32')
train_labels=train_labels.astype('int32')
test_data=test_data.astype('float32')
test_labels=test_labels.astype('int32')

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_1'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_2'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_3'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_4'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_5'>
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/test_batch'>


In [24]:
mean = np.mean(train_data,axis=(0,1,2,3))
std = np.std(train_data,axis=(0,1,2,3))
train_data = (train_data-mean)/(std+1e-7)

In [4]:
test_data = (test_data-mean)/(std+1e-7)

In [5]:
# datagen = ImageDataGenerator(
#     featurewise_center=False,
#     samplewise_center=False,
#     featurewise_std_normalization=False,
#     samplewise_std_normalization=False,
#     zca_whitening=False,
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=False
#     )
# datagen.fit(train_data)
# print(type(datagen))
# a = datagen.flow(train_data,train_labels, batch_size=55000)
# x,y = a.next()
# print(x.shape,y.shape,type(x),type(y))
# train_data = x
# train_labels = y

In [6]:
tf.logging.set_verbosity(tf.logging.ERROR)


def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])

    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv1_bn = tf.layers.batch_normalization(
        inputs=conv1,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True)
    conv1_bn_relu = tf.nn.relu(conv1_bn)

    conv2 = tf.layers.conv2d(
        inputs=conv1_bn_relu,
        filters=32,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv2_bn = tf.layers.batch_normalization(
        inputs=conv2,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv2_bn_relu = tf.nn.relu(conv2_bn)
    pool1 = tf.layers.max_pooling2d(
        inputs=conv2_bn_relu, pool_size=[2, 2], strides=2)

    dropout1 = tf.layers.dropout(
        inputs=pool1, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)

    conv3 = tf.layers.conv2d(
        inputs=dropout1,
        filters=64,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv3_bn = tf.layers.batch_normalization(
        inputs=conv3,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True)
    conv3_bn_relu = tf.nn.relu(conv3_bn)

    conv4 = tf.layers.conv2d(
        inputs=conv3_bn_relu,
        filters=64,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv4_bn = tf.layers.batch_normalization(
        inputs=conv4,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv4_bn_relu = tf.nn.relu(conv4_bn)
    pool2 = tf.layers.max_pooling2d(
        inputs=conv4_bn_relu, pool_size=[2, 2], strides=2)

    dropout2 = tf.layers.dropout(
        inputs=pool2, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

    conv5 = tf.layers.conv2d(
        inputs=dropout2,
        filters=128,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv5_bn = tf.layers.batch_normalization(
        inputs=conv5,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True)
    conv5_bn_relu = tf.nn.relu(conv5_bn)

    conv6 = tf.layers.conv2d(
        inputs=conv5_bn_relu,
        filters=128,
        kernel_size=[3, 3],
        padding="same",
        activation=None)
    conv6_bn = tf.layers.batch_normalization(
        inputs=conv6,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv6_bn_relu = tf.nn.relu(conv6_bn)
    pool3 = tf.layers.max_pooling2d(
        inputs=conv6_bn_relu, pool_size=[2, 2], strides=2)

    dropout3 = tf.layers.dropout(
        inputs=pool3, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

    flattened = tf.reshape(dropout3, [-1, 4 * 4 * 128])
    dense1 = tf.layers.dense(inputs=flattened, units=2048, activation=None)
    logits = tf.layers.dense(inputs=dense1, units=10)

    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy":
        tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]),
    }
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [7]:
def current_loss(flag):
    train_eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      shuffle=False)
    eval_result1=cifar_classifier.evaluate(input_fn=train_eval_input_fn)
    if flag:
        print("%%%%%%% Train accuracy %%%%%%%%%%%%\n",eval_result1)
    [a,b,c] = eval_result1.items()

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False)
    eval_result2=cifar_classifier.evaluate(input_fn=eval_input_fn)
    if flag:
        print("######### Test accuracy #############\n",eval_result2)
    [d,e,f] = eval_result2.items()
    return a[1],d[1]

In [8]:
# Create the Estimator
cifar_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="./cifar_90")

In [9]:
# # Set up logging for predictions
# # Log the values in the "Softmax" tensor with label "probabilities"
# tensors_to_log = {"probabilities": "softmax_tensor"}
# logging_hook = tf.train.LoggingTensorHook(
#   tensors=tensors_to_log, every_n_iter=50)

# # Train the model
# while True:
#     train_input_fn = tf.estimator.inputs.numpy_input_fn(
#       x={"x": train_data},
#       y=train_labels,
#       batch_size=64,
#       num_epochs=None,
#       shuffle=True)

#     cifar_classifier.train(
#       input_fn=train_input_fn,
#       steps= 1000,
#       hooks=None)
#     curr_result = current_loss(1)
# #     if curr_result[0] - curr_result[1] > 0.10 or curr_result[1] > 0.98:
# #        break
#     #print(curr_result)
# print ("*****************END*******************")

In [10]:
# current_loss(0)

In [11]:
test_data.shape

(10000, 32, 32, 3)

In [20]:
test = np.copy(test_data)
test = test.reshape(10000,1024,3)
print(test.shape)

(10000, 1024, 3)


In [21]:
for i in range(0,3):
    test[:,:,i] = SP.Scaling(test[:,:,i],0.9)

In [17]:
SP.Test(train_data,train_labels,test_data,test_labels,cifar_classifier)

%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.8247, 'loss': 0.5899375, 'global_step': 265003}
######### Test accuracy #############
 {'accuracy': 0.7927, 'loss': 0.73559433, 'global_step': 265003}


In [22]:
test = (test-mean)/(std+1e-7)

In [25]:
SP.Test(train_data,train_labels,test,test_labels,cifar_classifier)

%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.8247, 'loss': 0.5899375, 'global_step': 265003}
######### Test accuracy #############
 {'accuracy': 0.6634, 'loss': 1.341758, 'global_step': 265003}
